delete할 때, parser tree result를 이용해서 문장을 축약해보자.

In [4]:
import spacy
import numpy
import sense2vec
import random

from numpy import dot
from numpy.linalg import norm
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from data_handling_for_heuristic import *

stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
nlp = spacy.load('en')
#model = sense2vec.load()

import os
os.environ['STANFORD_PARSER'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser.jar'
os.environ['STANFORD_MODELS'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser-3.7.0-models.jar'
from nltk.parse.stanford import StanfordParser
cons_parser = StanfordParser(model_path = 'edu\\stanford\\nlp\\models\\lexparser\\englishPCFG.ser.gz')

# Load Dataset

In [2]:
# #read_path = 'data/conll2003/eng.train'
# read_path = 'train.txt'
# with open(read_path, "r") as ins:
#     raw_data = []
#     label_data = []
    
#     temp_sent = ''
#     temp_label = ''
#     tkn = False
    
#     for line in ins:
#         #array.append(line)

#         if len(line) == 1:
#             raw_data.append(temp_sent)
#             label_data.append(temp_label)
#             temp_sent = ''
#             temp_label = ''
#             tkn = False
#         else:
#             if tkn == True:
#                 temp_sent += ' '
#                 temp_label += ' '
                
#             temp_sent += line.split()[0] # 단어
#             temp_label += line.split()[-1] # NER 라벨
#             tkn = True

In [5]:
sent_raw = []
sent_label = []
cnt = 0
for i, row in enumerate(raw_data):
    row_nlp = nlp(row)

#     if len(raw_data[i].split()) >= 2: # 길이가 최소 2이상인 문장들마나 선별하자. (ex. .만 있는 문장도 있다)
#         temp = []
#         for token in row_nlp:
#             temp.append(token.pos_) # 각 token의 pos 저장
#         if 'VERB' in temp: # 문장에 최소 1개이상의 verb가 있어야 한다.
#             if len([x for x in label_data[i].split() if x != 'O']) != 0: # 엔티티가 하나도 없으면 안된다.
#                 sent_raw.append(raw_data[i])
#                 sent_label.append(label_data[i]) 
 
    
    
    if raw_data[i][-1] == '.' or raw_data[i][-1] == '"': # 마지막에 쉼표가 있는 문장들만 선별하자.
        if len(raw_data[i].split()) >= 2: # 길이가 최소 2이상인 문장들마나 선별하자. (ex. .만 있는 문장도 있다)
            temp = []
            for token in row_nlp:
                temp.append(token.pos_) # 각 token의 pos 저장
            if 'VERB' in temp: # 문장에 최소 1개이상의 verb가 있어야 한다.
                if len([x for x in label_data[i].split() if x != 'O']) != 0: # 엔티티가 하나도 없으면 안된다.
                    sent_raw.append(raw_data[i])
                    sent_label.append(label_data[i]) 
#                 else:
#                     cnt += 1

read_file_path = 'data/conll2003/train.txt'
raw_data, label_data = load_conll2003(read_file_path)

filter_none_entity = True
if filter_none_entity == True:
    sent_raw, sent_label = filtering_none_entity(raw_data, label_data)
else:
    sent_raw = raw_data[:]
    sent_label = label_data[:] 

[filtering_none_entity]: before = 14987 and after = 11132


sent_raw: 문장 단위 + 정상적인 문장 텍스트 <br>
sent_label: 문장 단위 + 정상적인 문장 텍스트

## 임시

In [6]:
# from data_handling_for_heuristic import *
# #read_path = 'data/conll2003/eng.train'
# read_path = 'data_logic_warehouse/type_2/merged_replace.txt'
# #read_path = 'train.txt'
# sent_raw, sent_label = load_conll2003(read_path)

In [7]:
len(sent_raw)

11132

In [8]:
### Preprocessing

def del_ent_in_list(delete_list, X):
# token 단위로 확인하면서 삭제한다
# call to ( boycott ) 이면 (과 )는 삭제된다. token단위이므로... (token단위 기준: space)
# X: ['EU rejects German call to boycott British lamb .', ... ]
# Y: ['I-ORG O I-MISC O O O I-MISC O O', ... ]
# delete_list = [')','(', ';', ...]

    i = 0
    while i < len(X):
        if any(str(X[i]) in t for t in delete_list):
            #del Y[i]
            del X[i]
        #else:
        i+=1               
    return X #, Y

def del_ent_in_string(delete_list, X):
# char 단위로 확인하면서 삭제한다
# 예를 들어, boycott이 아니라 boy(cott이면 해당 token은 삭제된다. 이처럼 char레벨까지 체크한다.
# X: ['EU rejects German call to boycott British lamb .', ... ]
# Y: ['I-ORG O I-MISC O O O I-MISC O O', ... ]
# delete_list = [')','(', ';', ...]    
    i=0
    while i < len(X):
        #print(str(sp_test[i]))
        list_X = list(X[i])
        if len(list_X)==1:
            break
        for char in list_X:
            if any(char in t for t in delete_list):
                #print(X)
                #print(Y)
                #del Y[i]
                del X[i]
                break
        #else:
        i+=1
    return X#, Y

In [9]:
global index_SBAR
global list_SBAR 

# index_SBAR = -1
# list_SBAR = [0] * len(sent_raw[3].split())

def traverseTree(tree, temp_sentence):

    global index_SBAR
    global list_SBAR    
    
#     if tree.label() == 'ROOT': # 처음에 initialization
#         index_SBAR = -1
#         list_SBAR = [0] * len(sent_list)

    #print("tree:", tree)
    #print("======> tree.pos()", tree.pos())
    #print("======> tree.height()", tree.height())
    #print("======> tree.label()", tree.label())
    #print("======> tree.leaves()", tree.leaves())
    
    #print("======> current_depth", current_depth)
    #print('======> current depth list', list_current_depth)

    #print('*************************************************************************************************')    
    #print('\n')

    if type(tree) == nltk.tree.Tree:
        
        # Initialication
        current_height = tree.height()

        
        ### 로직 한계점
        ### 만약 SBAR안에 SBAR이 있는 경우 구분하지 못한다. 또는 SBAR이 또다른 SBAR과 바로 옆에 붙어있는 경우 구분하지 못한다.
        ### 그러면 나중에 제일 첫 글자만 참고해서 어떤 부사절인지 보는데, 만약 2개 이상 SBAR이 포함되어있거나 붙어있는 경우
        ### index가 제일 앞에 있는 SBAR만 참고하게 된다. (예를 들어, because SBAR이 뒤에 있는 경우 무시하게 된다)
        
        # SBAR
        if tree.label() == 'SBAR':
            #print('SBAR start!!!')
            
            if len(tree.leaves()) <= 1: # SBAR token개수가 1개 이하이면 return. 
                return None
            
            for i, token in enumerate(temp_sentence):
                #print('temp_sentence[i]:', temp_sentence[i], ', tree.leaves()[0]:',tree.leaves()[0]) 
                #print('temp_sentence[i+1]:', temp_sentence[i+1], ', tree.leaves()[1]:', tree.leaves()[1])
                
                
                #if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+1] == tree.leaves()[1]:
                    #print('break!!!!!!!!!!!!!!!!!!!!!!')
                    index_SBAR = i
                    break
                
            if not index_SBAR == -1:
                for j in range(index_SBAR, index_SBAR+len(tree.leaves())):
                    list_SBAR[j] = 1   
                    
            return None

    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            traverseTree(subtree, temp_sentence) # recursive

In [10]:
def re_numbering(list_SBAR):
# ex. [1,1,1,0,0,1,1,0,1,1] => [1,1,1,0,0,2,2,0,3,3]
# 후처리하기 쉽게하기 위해 numbering을 다시한다.
    
    cur_idx = 1
    new_list_SBAR = [0] * len(list_SBAR)
    
    for i, _ in enumerate(list_SBAR):
        if list_SBAR[i]!=0:
            new_list_SBAR[i] = cur_idx
        if not i==len(list_SBAR)-1:
            if list_SBAR[i]!=0 and list_SBAR[i+1]==0:
                cur_idx += 1
    return new_list_SBAR

In [11]:
global cnt_MainClause_delete_SBARwithoutENT
global cnt_AdverbialClause_reuse_SBAR
global cnt_MainClause_delete_SBARwithENT

cnt_MainClause_delete_SBARwithoutENT = 0 # SBAR에 엔티티가 없는경우 그냥 삭제.
cnt_AdverbialClause_reuse_SBAR = 0 # SBAR이 완전한 문장이고,  그 안에 엔티티가 있는경우 재사용.
cnt_MainClause_delete_SBARwithENT = 0

def gen_with_SBAR(list_SBAR, token_raw_list, token_label_list):
    
    global cnt_MainClause_delete_SBARwithoutENT
    global cnt_AdverbialClause_reuse_SBAR
    global cnt_MainClause_delete_SBARwithENT
    
    
    # 주의: list_SBAR, sent_raw_list, sent_label_list의 길이는 모두 같다.
    
    sent_clauses = ['because', 'before', 'until', 'after', 'while', 'if', 'since', 'when', 'as', 'where',
                   'Because', 'Before', 'Until', 'After', 'While', 'If', 'Since', 'When', 'As'] 
    noun_clauses = ['which', 'who', 'to']
    # to는 to부정사를 지칭하는 거임. 좀 더 정확하게 하려면 다음에 동사인 것을 확인해야 하지만, SBAR인점과 첫 글자가 to이라는 것은 to부정사 확률이 높음.
    # 명사절에서 that절은 빼자. that이 워낙 경우의 수가 다양해서.. 일단은 배제하자.
    
    new_raw_sent_list = [] # 문장 여러개 
    new_label_sent_list = []
    
    #print(token_raw_list)
    #print(list_SBAR)
    #print('------------------------------------------')    

    for i, _ in enumerate(list_SBAR):
        temp_raw_str = ''
        temp_label_str = ''
        

        
        ##########################
        ###### @@ 로직 똑같음 ######
        ##########################
        if i==0 and list_SBAR[i] != 0: # 앞 boundary일 경우의 수 1
        ##########################    
        ##########################    
            if str(token_raw_list[i]) in sent_clauses: # 삭제, 분리
                
                # 엔티티가 해당 clause에 있는지 확인
                is_ent_in_clause = False
                for k, _ in enumerate(list_SBAR):
                    if list_SBAR[k]==list_SBAR[i]: # 해당 번호
                        if token_label_list[k]!='O': # 엔티티가 있다는 뜻
                            is_ent_in_clause = True
                            
                if is_ent_in_clause == False: # 엔티티가 없으면.. 해당 clause 삭제
                    for m, n in enumerate(list_SBAR):
                        if n != list_SBAR[i]: # clause를 삭제하자. 즉, clause에 속하지 않는 것들만 추리자.
                            temp_raw_str += str(token_raw_list[m])
                            temp_raw_str += ' '
                            temp_label_str += str(token_label_list[m])
                            temp_label_str += ' '
                            
                    ### clause가 삭제된 문장 저장
                    #print(temp_raw_str)
                    cnt_MainClause_delete_SBARwithoutENT += 1
                    new_raw_sent_list.append(temp_raw_str)
                    new_label_sent_list.append(temp_label_str)
                    
                if is_ent_in_clause == True: # 엔티티가 있으면..
                    fir_tkn = False
                    for m, n in enumerate(list_SBAR):
                        if n == list_SBAR[i]: # clause에 속하는 것들만 추리자
                            if fir_tkn == False: # 첫 번째 글자는 pass하자. (ex. because, if, while 등등)
                                fir_tkn = True
                                continue
                            temp_raw_str += str(token_raw_list[m])
                            temp_raw_str += ' '
                            temp_label_str += str(token_label_list[m])
                            temp_label_str += ' '
                            
                    ### clause가 삭제된 문장 저장
                    #print(temp_raw_str)
                    cnt_AdverbialClause_reuse_SBAR += 1
                    new_raw_sent_list.append(temp_raw_str)
                    new_label_sent_list.append(temp_label_str)
                    
                    # 엔티티가 clause외에도 있는지 확인
                    is_ent_out_clause = False
                    for k, _ in enumerate(list_SBAR):
                        if list_SBAR[k]!=list_SBAR[i]: # 해당 번호
                            if token_label_list[k]!='O': # 엔티티가 있다는 뜻
                                is_ent_out_clause = True                
                    
                    # 엔티티가 clause외에 있으면 그것도 데이터확장에 사용.
                    temp_raw_str = ''
                    temp_label_str = ''
                    if is_ent_out_clause == True:
                        for m, n in enumerate(list_SBAR):
                            if n != list_SBAR[i]: # clause에 속하지 않는 것들만 추리자.
                                temp_raw_str += str(token_raw_list[m])
                                temp_raw_str += ' '
                                temp_label_str += str(token_label_list[m])
                                temp_label_str += ' '

                        ### clause가 삭제된 문장 저장
                        #print(temp_raw_str)
                        cnt_MainClause_delete_SBARwithENT += 1
                        new_raw_sent_list.append(temp_raw_str)
                        new_label_sent_list.append(temp_label_str)                
                
                

            elif str(token_raw_list[i]) in noun_clauses: # 삭제만함.

                # 엔티티가 해당 clause에 있는지 확인
                is_ent_in_clause = False
                for k, _ in enumerate(list_SBAR):
                    if list_SBAR[k]==list_SBAR[i]: # 해당 번호
                        if token_label_list[k]!='O': # 엔티티가 있다는 뜻
                            is_ent_in_clause = True
                            
                if is_ent_in_clause == False: # 엔티티가 없으면.. 해당 clause 삭제
                    for m, n in enumerate(list_SBAR):
                        if n != list_SBAR[i]: # clause를 삭제하자. 즉, clause에 속하지 않는 것들만 추리자.
                            temp_raw_str += str(token_raw_list[m])
                            temp_raw_str += ' '
                            temp_label_str += str(token_label_list[m])
                            temp_label_str += ' '
                            
                    ### clause가 삭제된 문장 저장
                    #print(temp_raw_str)
                    cnt_MainClause_delete_SBARwithoutENT += 1
                    new_raw_sent_list.append(temp_raw_str)
                    new_label_sent_list.append(temp_label_str)                
                

        
                
        ##########################                
        ###### @@ 로직 똑같음 ######
        ##########################            
        if i != 0:
            if list_SBAR[i-1]==0 and list_SBAR[i] != 0: # 앞 boundary일 경우의 수 2
        ##########################
        ##########################
        
                if str(token_raw_list[i]) in sent_clauses: # 삭제, 분리


                    # 엔티티가 해당 clause에 있는지 확인
                    is_ent_in_clause = False
                    for k, _ in enumerate(list_SBAR):
                        if list_SBAR[k]==list_SBAR[i]: # 해당 번호
                            if token_label_list[k]!='O': # 엔티티가 있다는 뜻
                                is_ent_in_clause = True

                    if is_ent_in_clause == False: # 엔티티가 없으면.. 해당 clause 삭제
                        for m, n in enumerate(list_SBAR):
                            if n != list_SBAR[i]: # clause를 삭제하자. 즉, clause에 속하지 않는 것들만 추리자.
                                temp_raw_str += str(token_raw_list[m])
                                temp_raw_str += ' '
                                temp_label_str += str(token_label_list[m])
                                temp_label_str += ' '

                        ### clause가 삭제된 문장 저장
                        #print(temp_raw_str)
                        cnt_MainClause_delete_SBARwithoutENT += 1
                        new_raw_sent_list.append(temp_raw_str)
                        new_label_sent_list.append(temp_label_str)

                    if is_ent_in_clause == True: # 엔티티가 있으면..
                        fir_tkn = False
                        for m, n in enumerate(list_SBAR):
                            if n == list_SBAR[i]: # clause에 속하는 것들만 추리자
                                if fir_tkn == False: # 첫 번째 글자는 pass하자. (ex. because, if, while 등등)
                                    fir_tkn = True
                                    continue
                                temp_raw_str += str(token_raw_list[m])
                                temp_raw_str += ' '
                                temp_label_str += str(token_label_list[m])
                                temp_label_str += ' '

                        ### clause가 삭제된 문장 저장
                        #print(temp_raw_str)
                        cnt_AdverbialClause_reuse_SBAR += 1
                        new_raw_sent_list.append(temp_raw_str)
                        new_label_sent_list.append(temp_label_str)

                        # 엔티티가 clause외에도 있는지 확인
                        is_ent_out_clause = False
                        for k, _ in enumerate(list_SBAR):
                            if list_SBAR[k]!=list_SBAR[i]: # 해당 번호
                                if token_label_list[k]!='O': # 엔티티가 있다는 뜻
                                    is_ent_out_clause = True                

                        # 엔티티가 clause외에 있으면 그것도 데이터확장에 사용.
                        temp_raw_str = ''
                        temp_label_str = ''
                        if is_ent_out_clause == True:
                            for m, n in enumerate(list_SBAR):
                                if n != list_SBAR[i]: # clause에 속하지 않는 것들만 추리자.
                                    temp_raw_str += str(token_raw_list[m])
                                    temp_raw_str += ' '
                                    temp_label_str += str(token_label_list[m])
                                    temp_label_str += ' '

                            ### clause가 삭제된 문장 저장
                            #print(temp_raw_str)
                            cnt_MainClause_delete_SBARwithENT += 1
                            new_raw_sent_list.append(temp_raw_str)
                            new_label_sent_list.append(temp_label_str)                



                elif str(token_raw_list[i]) in noun_clauses: # 삭제만함.

                    # 엔티티가 해당 clause에 있는지 확인
                    is_ent_in_clause = False
                    for k, _ in enumerate(list_SBAR):
                        if list_SBAR[k]==list_SBAR[i]: # 해당 번호
                            if token_label_list[k]!='O': # 엔티티가 있다는 뜻
                                is_ent_in_clause = True

                    if is_ent_in_clause == False: # 엔티티가 없으면.. 해당 clause 삭제
                        for m, n in enumerate(list_SBAR):
                            if n != list_SBAR[i]: # clause를 삭제하자. 즉, clause에 속하지 않는 것들만 추리자.
                                temp_raw_str += str(token_raw_list[m])
                                temp_raw_str += ' '
                                temp_label_str += str(token_label_list[m])
                                temp_label_str += ' '

                        # clause가 삭제된 문장 저장
                        #print(temp_raw_str)
                        cnt_MainClause_delete_SBARwithoutENT += 1
                        new_raw_sent_list.append(temp_raw_str)
                        new_label_sent_list.append(temp_label_str)                
                
    
    return new_raw_sent_list, new_label_sent_list

# Main

In [12]:
# 괄호가 있어도 상관은 없는데..
# tree.leaves()에서는 괄호는 출력되지 않는다.
# 하지만1, string안에 괄호가 있는 경우는 parser 오류가 난다. 예를 들어 ap(ple인 단어가 있으면 오류가 난다. 
# 하지만2, 괄호가 있지만, 쌍으로 존재하지 않을경우 parser 오류가 난다. 예를 들어, ), ( 이렇게 한 개씩만 있는 경우...
# 즉, 한 문장에서 괄호가 pair로 존재하지 않는 경우는 삭제하자.!!!

In [13]:
test = [
'While', 
'57',
'percent',
'of',
'the',
'population',
'supported',
'negotiations',
'with',
'ETA',
'(',
'Basque',
'Homeland',
'and',
'Freedom',
')',
',',
'30',
'percent',
'opposed',
'it',
',',
'the',
'survey',
'by',
'the',
'state-controlled',
'Centre',
'for',
'Sociological',
'Studies',
'(',
'CIS',
')',
'found',
'.']

In [14]:
test = [
'British', 
'Jewish', 
'groups', 
'have', 
'also', 
'voiced', 
'protest', 
'because', 
'they', 
'said', 
'Palestinian', 
'Islamist',
'Hamas',
'as', 
'well',
'as',
'the',
'banned',
'Algerian',
'Islamic',
'Salvation',
'Front',
'(',
'FIS',
')',
'are',
'among',
'those',
'radical',
'Islamists',
'attending',
'the',
'conference',
'.']

In [23]:
new_sent_list = []
new_label_list = [] 

for i, _ in enumerate(sent_raw):
    
#     print('***********************************')
#     print(sent_raw[i])
    
    token_raw_list = sent_raw[i].split()
    #token_raw_list = test
    token_label_list = sent_label[i].split()
    
    #print(test)
    
    
    #print(sent_raw[i])
    #print(sent_label[i])
    
    if '(' in token_raw_list:
        continue
    if ')' in token_raw_list:
        continue
    
    
    ### 괄호 처리
#     parenthesis_list = []
#     for i, _ in enumerate(token_raw_list):
#         if token_raw_list[i]=='(' or token_raw_list[i]==')':
#             parenthesis_list.append((token_raw_list[i], i)) # append (pair)

#     ### preprocessing
#     pre_raw_list = del_ent_in_list(['(',')'], token_raw_list)
#     pre2_raw_list = del_ent_in_string(['(',')'], token_raw_list) # 한 문장을 위해 실시.
    
    #print(pos_sent_raw_list)
    
    
    
    ### pos tagging
    pos_sent_raw_list = nltk.pos_tag(token_raw_list)
    #pos_sent_raw_list = nltk.pos_tag(token_raw_list)
    ### parsing
    parse_result = cons_parser.tagged_parse(pos_sent_raw_list)

    ###################################
    """ SBAR, 부사절 삭제 및 분리 """
    ###################################
    # initialization
    index_SBAR = -1
    list_SBAR = [0] * len(token_raw_list)    
    
    for tree in parse_result:
        parse_tree = tree
        traverseTree(parse_tree, token_raw_list)# tree traverse하면서 SBAR tree label을 찾음.
    
    #print(list_SBAR)
    
    ###################################
    """ 괄호 복구 작업 """
    ###################################
#     for pair in parenthesis_list: # 순서대로 리스트업 했기 때문에 순서대로 insert하기만 하면 됨. 
#         parenthesis = pair[0]
#         index = pair[1]
#         # token_raw_list에 괄호 복구
#         token_raw_list.insert(index, parenthesis)
#         # SBAR list에 괄호 복구
#         if parenthesis == '(':
#             if list_SBAR[index+1] == 1:
#                 list_SBAR.insert(index, 1)
#             else:
#                 list_SBAR.insert(index, 0)
#         elif parenthesis == ')':
#             if list_SBAR[index-1] == 1:
#                 list_SBAR.insert(index, 1)
#             else:
#                 list_SBAR.insert(index, 0)
            
    ##############################################
    """ Boundary 검사: 엔티티가 쪼개져 있으면 수정 """    
    ##############################################
    # parsing 결과를 보정
    tkn = True
    while(tkn):
        
        btn = False
        for i, num in enumerate(list_SBAR):
            if i!=len(list_SBAR)-1:
                if list_SBAR[i] == 1 and list_SBAR[i+1] == 0:
                    btn = True
        if btn == False and list_SBAR[-1] == 1:
            tkn = False

        if not 1 in list_SBAR:
            tkn = False
            
        for i, num in enumerate(list_SBAR):
            if i!=0: # 첫 번째 index는 제외
                if list_SBAR[i-1] == 1 and list_SBAR[i] == 0: # find SBAR boundary!
                    if token_label_list[i-1] != 'O': # 엔티티라면
                        if token_label_list[i].split('-')[0] == 'I': # (좀 더 구체적으로..) INSIDE 계열의 엔티티라면..
                            list_SBAR[i] = 1
                        else: # BEGIN 계열이면 그냥 pass
                            tkn = False # break
                    else: # 엔티티가 아니라면
                        tkn = False # break

    #print('hi')
    
                        
    ### for printf 
#     print(list_SBAR)
#     if 1 in list_SBAR:
#         temp = ''
#         for k, tkn in enumerate(token_raw_list):
#             if list_SBAR[k] == 1:
#                 temp = temp+token_raw_list[k]+' '
#         print(temp)
              
            
            
    # generation with SBAR
    if 1 in list_SBAR: # 현재 문장에서 SBAR이 있으면...
        # renumbering
        list_SBAR = re_numbering(list_SBAR)
        new_sent_raw_list, new_sent_label_list = gen_with_SBAR(list_SBAR, token_raw_list, token_label_list)
        #print(new_sent_raw_list)
        #print('\n')
        
        if len(new_sent_raw_list) != 0: # 새로운 문장이 생성되었는가?
            # 주의: append가 아니라 +로해야됨. new_sent_list가 list이기 때문에
            new_sent_list += new_sent_raw_list
            new_label_list += new_sent_label_list
            
    # 엔티티 포함되어 있으면 냅둔다.

#     for i, _ in enumerate(list_SBAR):

#         if not i==0:
#             if list_SBAR[i-1]==0 and list_SBAR[i]==1:

#         else: # i==0일 때...

        #print('------------------------------------------------')            
        #print(sent_raw[i])
        #print('------------------------------------------------')
        #print(temp)
        #print(list_SBAR)
        #print('------------------------------------------------')
    #print('\n')
    
    #break

KeyboardInterrupt: 

In [13]:
print(len(new_sent_list), len(new_label_list))

2054 2054


In [14]:
print(cnt_MainClause_delete_SBARwithoutENT)
print(cnt_AdverbialClause_reuse_SBAR)
print(cnt_MainClause_delete_SBARwithENT)

777
717
560


# Store new sent

In [15]:
len(new_sent_list)

2054

In [16]:
path_write = 'data_logic_warehouse/type_2/merged_replace_delete_'+str(len(new_sent_list))+'.txt'

with open(path_write, 'w', encoding='UTF-8') as txt:
    
    for i, _ in enumerate(new_sent_list):
        splited_sent = new_sent_list[i].split()
        splited_label = new_label_list[i].split()
        for j, token in enumerate(splited_sent):
            txt.write(splited_sent[j]+' '+'NNP'+' '+'B-NP'+' '+splited_label[j])
            txt.write('\n')
        txt.write('\n')

In [ ]:
# ### 테스트용
# test = 'Stanley owns a .367 career batting average with the bases loaded ( 33-for-90 ) because I have coffee .'
# test = 'The tournament also lost its second seed on the third day of play when second-seeded Goran Ivanisevic of Croatia was beaten 6-7(3-7 6-4 6-4 by unseeded Mikael Tillstrom of Sweden .'
# token_test_list = test.split()
# pos_token_test_list = nltk.pos_tag(token_test_list)
# parse_result = cons_parser.tagged_parse(pos_token_test_list)

# global index_SBAR
# global list_SBAR 

# index_SBAR = -1
# list_SBAR = [0] * len(test.split())

# def traverseTree1(tree, temp_sentence):

#     global index_SBAR
#     global list_SBAR 
# #     if tree.label() == 'ROOT': # 처음에 initialization
# #         index_SBAR = -1
# #         list_SBAR = [0] * len(sent_list)
#     #print("======> temp_sentence", temp_sentence)
#     #print("tree:", tree)
# #     print("======> tree.pos()", tree.pos())
# #     print("======> tree.height()", tree.height())
# #     print("======> tree.label()", tree.label())
#     #print("======> tree.leaves()", tree.leaves())
    
#     #print("======> current_depth", current_depth)
#     #print('======> current depth list', list_current_depth)

# #     print('*************************************************************************************************')    
# #     print('\n')

#     if type(tree) == nltk.tree.Tree:
        
#         # Initialication
#         current_height = tree.height()

#         # SBAR
#         if tree.label() == 'SBAR':
#             print("tree:", tree)
#             print(tree.leaves())
#             for i, token in enumerate(temp_sentence):
#                 print(i, temp_sentence[i], tree.leaves()[0], '<->', temp_sentence[i+len(tree.leaves())-1], tree.leaves()[-1])
#                 #if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:  
#                 if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+1] == tree.leaves()[1]:  
                    
#                     index_SBAR = i
#                     print('**********************************************************')
#                     break  
                
#             if not index_SBAR == -1:
#                 for j in range(index_SBAR, index_SBAR+len(tree.leaves())):
#                     list_SBAR[j] = 1 
        
  

#     for subtree in tree:
#         if type(subtree) == nltk.tree.Tree:
#             traverseTree1(subtree, temp_sentence) # recursive
            
# for tree in parse_result:
#     parse_tree = tree
#     traverseTree1(parse_tree, token_test_list)

In [ ]:
# if using_parsing == 1:
#     #########################
#     """ For training data """
#     #########################
#     cnt = 0
#     temp_depth_list = []
#     temp_n_sibling_list = []
#     for i, sentence in enumerate(pre_X_train):
#         temp = nltk.pos_tag(sentence)
#         par_result = parser.tagged_parse(temp)
#         for tree in par_result:
#             parse_tree = tree
#         #print(parse_tree)

#         # For PP, NP, VP,...
#         temp_sentence = sentence
#         initial_tree_parameter(sentence)
#         traverseTree(parse_tree) # For getting tree information

#         # For depth, n_sibling...
#         depth_list, n_siblings_list = tree_depth_and_n_sibling(parse_tree)
#         assert(len(sentence) == len(depth_list))
#     #     print(sentence)
#     #     print(parse_tree.leaves())
#         normalized_li_depth = normalizing_depth_list(depth_list)
#         normalized_li_n_siblings = normalizing_n_siblings_list(n_siblings_list)

#         for j, token in enumerate(sentence):
#             ### Tree depth and sibling
#             FE2_X_train[i][j] += FE_depth(j, sentence, normalized_li_depth, window_size_depth) 
#             FE2_X_train[i][j] += FE_n_siblings(j, sentence, normalized_li_n_siblings, window_size_n_siblings)     

#             ### Phrases-based Features
# #             FE2_X_train[i][j] += [list_NPSBAR[j]] # defalut 오직 SBAR만 뽑는다.
# #             FE2_X_train[i][j] += [list_VPSBAR[j]]
# #             FE2_X_train[i][j] += [list_SSBAR[j]]
# #             FE2_X_train[i][j] += [list_SQSBAR[j]]

# #             FE2_X_train[i][j] += [list_PPPP[j]] # default 오직 PP만 뽑는다.
# #             FE2_X_train[i][j] += [list_VPPP[j]]
# #             FE2_X_train[i][j] += [list_NPPP[j]] # sub_refinement_of_object를 추출하는데 도움..

# #             FE2_X_train[i][j] += [list_NPNP[j]]
# #             FE2_X_train[i][j] += [list_VPNP[j]] 
# #             FE2_X_train[i][j] += [list_SNP[j]] 

# #             FE2_X_train[i][j] += [list_VPVP[j]] 
# #             FE2_X_train[i][j] += [list_SVP[j]]      

        
#     #     cnt += 1
#     #     print('===================================================================')
#     #     if cnt == 1:
#     #         break

#     ########################
#     """ For testing data """
#     ########################
#     cnt = 0
#     for i, sentence in enumerate(pre_X_test):
#         temp = nltk.pos_tag(sentence)
#         par_result = parser.tagged_parse(temp)
#         for tree in par_result:
#             parse_tree = tree
#         #print(parse_tree)

#         # For PP, NP, VP, ...
#         temp_sentence = sentence
#         initial_tree_parameter(sentence)
#         traverseTree(parse_tree) # For getting tree information

#         # For dpeth, n_sibling
#         depth_list, n_siblings_list = tree_depth_and_n_sibling(parse_tree)
#         assert(len(sentence) == len(depth_list))
#     #     print(sentence)
#     #     print(parse_tree.leaves())
#         normalized_li_depth = normalizing_depth_list(depth_list)
#         normalized_li_n_siblings = normalizing_n_siblings_list(n_siblings_list)

#         for j, token in enumerate(sentence):

#             ### Tree-depth, Sibling 
#             FE2_X_test[i][j] += FE_depth(j, sentence, normalized_li_depth, window_size_depth) 
#             FE2_X_test[i][j] += FE_n_siblings(j, sentence, normalized_li_n_siblings, window_size_n_siblings) 

#             ## Phrases-based Festures
# #             FE2_X_test[i][j] += [list_NPSBAR[j]]
# #             FE2_X_test[i][j] += [list_VPSBAR[j]]
# #             FE2_X_test[i][j] += [list_SSBAR[j]]
# #             FE2_X_test[i][j] += [list_SQSBAR[j]]

# #             FE2_X_test[i][j] += [list_PPPP[j]]
# #             FE2_X_test[i][j] += [list_VPPP[j]]
# #             FE2_X_test[i][j] += [list_NPPP[j]] 

# #             FE2_X_test[i][j] += [list_NPNP[j]]
# #             FE2_X_test[i][j] += [list_VPNP[j]] 
# #             FE2_X_test[i][j] += [list_SNP[j]] 

# #             FE2_X_test[i][j] += [list_VPVP[j]] 
# #             FE2_X_test[i][j] += [list_SVP[j]]



#     #     print(convert_dummy2(depth_list[j], 0))
#     #     print(convert_dummy2(n_siblings_list[j], 1))
#     #     print([list_SBAR[j]])
#     #     print([list_VPPP[j]])
#     #     print([list_NPPP[j]])

#     #     cnt += 1
#     #     print('===================================================================')
#     #     if cnt == 1:
#     #         break
       
# #     print('===> current feature length', len(FE2_X_train[0][0])-len_pos_feature-len_chunk_feature)
# #     len_parsing_feature = len(FE2_X_train[0][0])-len_pos_feature-len_chunk_feature
# #     assert(len(FE2_X_train[0][0]) == len(FE2_X_test[0][0]))